### In this chapter we created own deep learning framework  
каждый тензор получает два новых атрибута. 

**creators** — это список любых тензоров, использовавшихся для создания текущего тензора (по умолчанию имеет значение None).То есть если тензор z получается сложением двух других тензоров, x и y, атрибут creators тензора z будет содержать тензоры x и y

**creation_op** - опутствующий атрибут, который хранит операции, использовавшиеся в процессе создания данного тензора. *То есть инструкция z = x + y создаст вычислительный граф с тремя узлами (x, y и z) и двумя ребрами (z -> x и z -> y). Каждое ребро при этом подписано операцией из creation_op, то есть add.* Этот граф поможет организовать рекурсивное обратное распространение градиентов.


In [27]:
import numpy as np

class Tensor(object):
    
    
    def __init__(self, data, creators=None, creation_op=None):
        self.data = np.array(data)
        self.creators = creators
        self.creation_op = creation_op
        self.grad = None
    
    # Автоматическое вычисление градиента
    def backward(self, grad):
        self.grad = grad
        if (self.creation_op == 'add'):
            self.creators[0].backward(grad)
            self.creators[1].backward(grad)
        
    
    def __add__(self, other):
        return Tensor(self.data + other.data,
                        creators=[self, other],
                        creation_op='add')
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())

Tensor — возможность использовать граф для вычисления градиентов. **Если вызвать метод z.backward()**, он передаст градиент для x и y с учетом функции, с помощью которой создавался тензор z (add). Как показано в примере выше, мы передаем вектор градиентов (np.array([1,1,1,1,1])) в z, а тот применяет его к своим родителям. Самой замечательной особенностью этой формы автоматического вычисления градиента является то, что она работает рекурсивно — каждый вектор вызывает метод .backward() всех своих родителей из списка self.creator. 

In [28]:
x = Tensor([1, 2, 3, 4, 5])
y = Tensor([2, 2, 2, 2, 2])
z = x + y
z.backward(Tensor(np.array([1, 1, 1, 1, 1])))

# В данном случае у нас только один градиент для добавления в x and y
print(x.grad)
print(y.grad)
print(z.creators)
print(z.creation_op)

[1 1 1 1 1]
[1 1 1 1 1]
[array([1, 2, 3, 4, 5]), array([2, 2, 2, 2, 2])]
add


In [29]:
a = Tensor([1,2,3,4,5])
b = Tensor([2,2,2,2,2])
c = Tensor([5,4,3,2,1])

d = a + b
e = b + c
f = d + e
f.backward(Tensor(np.array([1,1,1,1,1])))

b.grad.data == np.array([2,2,2,2,2])

array([False, False, False, False, False])

# Много кратное использование тензоров
Выше реализация имеет ошибку, нам нужно, что в случае многократного использования тензоров,  при накапливание градиента она получала градиент от всех потомков.

* Дополнительно **мы создали счетчик self.children, подсчитывающий количество градиентов, полученных от каждого потомка в процессе обратного распространения.** Так мы исключаем возможность случайного повторного обратного распространения от одного и того же потомка (в этом случае возбуждается исключение).
*  **all_children_grads_accounted_for(). Эта функция проверяет, были ли получены градиенты от всех потомков в графе.**
* Обычно, когда метод .backward() вызывается промежуточной переменной в графе, она тут же вызывает метод .backward() своего родителя. Но так как некоторые переменные получают свои градиенты от нескольких родителей, каждая должна отложить вызов .backward() своего родителя, пока не получит свой окончательный градиент.

In [ ]:
class Tensor (object):
    
    def __init__(self,data,
                 autograd=False,
                 creators=None,
                 creation_op=None,
                 id=None):
        
        self.data = np.array(data)
        self.autograd = autograd
        self.grad = None
        if(id is None):
            self.id = np.random.randint(0,100000)
        else:
            self.id = id
        
        self.creators = creators
        self.creation_op = creation_op
        self.children = {}
        
        if(creators is not None):
            for c in creators:
                if(self.id not in c.children): # Скорректировать число потомков данного тезнора
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1
    
    # Проверка на получили ли тензоры градиенты от всех потомков
    def all_children_grads_accounted_for(self):
        for id, cnt in self.children.items():
            if(cnt != 0):
                return False
        return True        
        
    def backward(self,grad=None, grad_origin=None):
        if(self.autograd):
            if(grad is None):
                grad = FloatTensor(np.ones_like(self.data))
            # Проверка возможности обратного распространения или ожидания градиента
            if(grad_origin is not None):
                if(self.children[grad_origin.id] == 0):
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1

            if(self.grad is None):
                self.grad = grad # Накопление градиента от нескольких потомков
            else:
                self.grad += grad
            
            # grads must not have grads of their own
            assert grad.autograd == False
            
            # only continue backpropping if there's something to
            # backprop into and if all gradients (from children)
            # are accounted for override waiting for children if
            # "backprop" was called on this variable directly
            if(self.creators is not None and 
               (self.all_children_grads_accounted_for() or 
                grad_origin is None)):

                if(self.creation_op == "add"):
                    # Фактически начало обратного распространения
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)
                    
    def __add__(self, other):
        # Включаем автограф
        if(self.autograd and other.autograd):
            return Tensor(self.data + other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="add")
        return Tensor(self.data + other.data)

    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())  

In [26]:
a = Tensor([1,2,3,4,5], autograd=True)
b = Tensor([2,2,2,2,2], autograd=True)
c = Tensor([5,4,3,2,1], autograd=True)

d = a + b
e = b + c
f = d + e

f.backward(Tensor(np.array([1,1,1,1,1])))

print(b.grad.data == np.array([2,2,2,2,2]))


[ True  True  True  True  True]


### Сложение в обратном распространении
А так выполняется обратное распространение градиента через функцию сложения в методе .backward():

In [2]:
# if(self.creation_op == "add"):
#   self.creators[0].backward(self.grad, self)
#   self.creators[1].backward(self.grad, self)

Обратите внимание, что сложение нигде в классе не обрабатывается. Логика обратного распространения абстрагирована, поэтому все необходимое для поддержки сложения определяется в этих двух местах. **Также обратите внимание, что логика обратного распространения вызывает .backward() дважды, по одному разу для каждой переменной, участвовавшей в сложении. То есть логика по умолчанию обратного распространения заключается в обратном распространении через каждую переменную в графе.**

# Добавление отрицания

In [9]:
import numpy as np
class Tensor (object):
    
    def __init__(self,data,
                 autograd=False,
                 creators=None,
                 creation_op=None,
                 id=None):
        
        self.data = np.array(data)
        self.autograd = autograd
        self.grad = None
        if(id is None):
            self.id = np.random.randint(0,100000)
        else:
            self.id = id
        
        self.creators = creators
        self.creation_op = creation_op
        self.children = {}
        
        if(creators is not None):
            for c in creators:
                if(self.id not in c.children): # Скорректировать число потомков данного тезнора
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1
    
    # Проверка на получили ли тензоры градиенты от всех потомков
    def all_children_grads_accounted_for(self):
        for id, cnt in self.children.items():
            if(cnt != 0):
                return False
        return True        
        
    def backward(self,grad=None, grad_origin=None):
        if(self.autograd):
            if(grad is None):
                grad = FloatTensor(np.ones_like(self.data))
            # Проверка возможности обратного распространения или ожидания градиента
            if(grad_origin is not None):
                if(self.children[grad_origin.id] == 0):
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1

            if(self.grad is None):
                self.grad = grad # Накопление градиента от нескольких потомков
            else:
                self.grad += grad
            
            # grads must not have grads of their own
            assert grad.autograd == False
            
            # only continue backpropping if there's something to
            # backprop into and if all gradients (from children)
            # are accounted for override waiting for children if
            # "backprop" was called on this variable directly
            if(self.creators is not None and 
               (self.all_children_grads_accounted_for() or 
                grad_origin is None)):

                if(self.creation_op == "add"):
                    # Фактически начало обратного распространения
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)
                    
                if(self.creation_op == "neg"):
                    # Поскольку в операции отрицания __neg__ участвует только один родитель, 
                    # метод .backward() должен вызываться только один раз.
                    self.creators[0].backward(self.grad.__neg__())
    
    # Сложение двух тензоров           
    def __add__(self, other):
        # Включаем автограф
        if(self.autograd and other.autograd):
            return Tensor(self.data + other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="add")
        return Tensor(self.data + other.data)
    
    # Операция меняет знак на минус 
    def __neg__(self):
        if (self.autograd):
            return Tensor(-self.data,
                         autograd=True,
                         creators=[self],
                         creation_op="neg")
        return Tensor(-self.data)

    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())  

In [10]:
a = Tensor([1,2,3,4,5], autograd=True)
b = Tensor([2,2,2,2,2], autograd=True)
c = Tensor([5,4,3,2,1], autograd=True)

d = a + (-b)
e = (-b) + c
f = d + e

f.backward(Tensor(np.array([1,1,1,1,1])))

print(b.grad.data == np.array([-2,-2,-2,-2,-2]))

[ True  True  True  True  True]


## Добавление других ариф операций 

In [11]:
import numpy as np
class Tensor (object):
    
    def __init__(self,data,
                 autograd=False,
                 creators=None,
                 creation_op=None,
                 id=None):
        
        self.data = np.array(data)
        self.autograd = autograd
        self.grad = None
        if(id is None):
            self.id = np.random.randint(0,100000)
        else:
            self.id = id
        
        self.creators = creators
        self.creation_op = creation_op
        self.children = {}
        
        if(creators is not None):
            for c in creators:
                if(self.id not in c.children): # Скорректировать число потомков данного тезнора
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1
    
    # Проверка на получили ли тензоры градиенты от всех потомков
    def all_children_grads_accounted_for(self):
        for id, cnt in self.children.items():
            if(cnt != 0):
                return False
        return True        
        
    def backward(self,grad=None, grad_origin=None):
        if(self.autograd):
            if(grad is None):
                grad = FloatTensor(np.ones_like(self.data))
            # Проверка возможности обратного распространения или ожидания градиента
            if(grad_origin is not None):
                if(self.children[grad_origin.id] == 0):
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1

            if(self.grad is None):
                self.grad = grad # Накопление градиента от нескольких потомков
            else:
                self.grad += grad
            
            # grads must not have grads of their own
            assert grad.autograd == False
            
            # only continue backpropping if there's something to
            # backprop into and if all gradients (from children)
            # are accounted for override waiting for children if
            # "backprop" was called on this variable directly
            if(self.creators is not None and 
               (self.all_children_grads_accounted_for() or 
                grad_origin is None)):

                if(self.creation_op == "add"):
                    # Фактически начало обратного распространения
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)
                    
                if(self.creation_op == "neg"):
                    # Поскольку в операции отрицания __neg__ участвует только один родитель, 
                    # метод .backward() должен вызываться только один раз.
                    self.creators[0].backward(self.grad.__neg__())
                    
                if(self.creation_op == "sub"):
                    self.creators[0].backward(self.grad)
                    
                if(self.creation_op == "mul"):
                    new = self.grad * self.creators[1]
                    self.creators[0].backward(new , self)
                    new = self.grad * self.creators[0]
                    self.creators[1].backward(new, self)                    
                    
                if(self.creation_op == "mm"):
                    c0 = self.creators[0]
                    c1 = self.creators[1]
                    new = self.grad.mm(c1.transpose())
                    c0.backward(new)
                    new = self.grad.transpose().mm(c0).transpose()
                    c1.backward(new)
                    
                if(self.creation_op == "transpose"):
                    self.creators[0].backward(self.grad.transpose())

                if("sum" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.expand(dim,
                                                               self.creators[0].data.shape[dim]))

                if("expand" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.sum(dim))
                         
    # Сложение двух тензоров           
    def __add__(self, other):
        # Включаем автограф
        if(self.autograd and other.autograd):
            return Tensor(self.data + other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="add")
        return Tensor(self.data + other.data)
    
    # Операция меняет знак на минус 
    def __neg__(self):
        if (self.autograd):
            return Tensor(-self.data,
                         autograd=True,
                         creators=[self],
                         creation_op="neg")
        return Tensor(-self.data)
    
    # Вычитание
    def __sub__(self, other):
        if (self.autograd and other.autograd):
            return Tensor(self.data - other.data,
                         autograd=True,
                         creators=[self, other],
                         creation_op="sub")
        return Tensor(self.data - other.data)
    
    # Умножение 
    def __mul__(self, other):
        if (self.autograd and other.autograd):
            return Tensor(self.data * other.data,
                         autograd=True,
                         creators=[self, other],
                         creation_op="mul")
        return Tensor(self.data * other.data)
    
    # Функция sum выполняет сложение элементов тензора по измерениям
    def sum(self, dim):
        if (self.autograd):
            return Tensor(self.data.sum(dim),
                         autograd=True,
                         creators=[self],
                         creation_op="sum_" + str(dim))
        return Tensor(self.data.sum(dim))
    
    # Функция expand используется для обратного распространения операции .sum(). 
    # Она копирует данные по измерению.
    def expand(self, dim, copies):
        
        trans_cmd = list(range(0,len(self.data.shape)))
        trans_cmd.insert(dim,len(self.data.shape))
        new_shape = list(self.data.shape) + [copies]
        new_data = self.data.repeat(copies).reshape(new_shape)
        new_data = new_data.transpose(trans_cmd)

        if(self.autograd):
            return Tensor(new_data,
                          autograd=True,
                          creators=[self],
                          creation_op="expand_"+str(dim))
        return Tensor(new_data)
    
    def transpose(self):
        if(self.autograd):
            return Tensor(self.data.transpose(),
                          autograd=True,
                          creators=[self],
                          creation_op="transpose")
        return Tensor(self.data.transpose())
    
    def mm(self, x):
        if(self.autograd):
            return Tensor(self.data.dot(x.data),
                          autograd=True,
                          creators=[self,x],
                          creation_op="mm")
        return Tensor(self.data.dot(x.data)) 

    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())  

В конце сети вычисляются градиенты, после чего сигнал ошибки начинает распространяться через сеть в обратном направлении вызовом функций, соответствующих функциям, которые использовались для прямого распространения активаций. **Если последней была операция умножения матриц (а это так и есть), в обратном распространении производится умножение матрицы (dot) на транспонированную матрицу.**

На следующем рисунке это происходит **в строке layer_1_delta=layer_2_delta.dot (weights_1_2.T). В предыдущем коде это происходит в инструкции if(self.creation_op == "mm")** (соответствующие строки выделены жирным). Здесь мы выполняем те же операции, что и прежде (в порядке, обратном порядку прямого распространения), но код организован гораздо лучше.

Отрывок из книги: Траск Э. «Грокаем глубокое обучение». Apple Books. 

# Записи о SUM and Expand

In [13]:
x = Tensor(np.array([[1,2,3],
                     [4,5,6]]))

In [14]:
# Number it's demension of summary 
x.sum(0)

array([5, 7, 9])

In [15]:
x.sum(1)

array([ 6, 15])

In [25]:
x.expand(dim=2, copies=5)

array([[[1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3]],

       [[4, 4, 4, 4, 4],
        [5, 5, 5, 5, 5],
        [6, 6, 6, 6, 6]]])

## Используем наш фреймворк и автограф для тренировки 

In [12]:
import numpy
np.random.seed(0)

data = Tensor(np.array([[0,0],[0,1],[1,0],[1,1]]), autograd=True)
target = Tensor(np.array([[0],[1],[0],[1]]), autograd=True)

w = list()
w.append(Tensor(np.random.rand(2,3), autograd=True))
w.append(Tensor(np.random.rand(3,1), autograd=True))

for i in range(10):

    # Predict
    pred = data.mm(w[0]).mm(w[1])
    
    # Compare
    loss = ((pred - target)*(pred - target)).sum(0)
    
    # Learn
    loss.backward(Tensor(np.ones_like(loss.data)))

    for w_ in w:
        w_.data -= w_.grad.data * 0.1
        w_.grad.data *= 0

    print(loss)

[0.58128304]
[0.42945783]
[0.31080708]
[0.22343058]
[0.1777608]
[0.16803502]
[0.25763598]
[0.32192563]
[0.61786922]
[0.37022632]
